In [87]:

def scrub_text_column(to_clean):
    
    '''This cleans text columns by stripping whitespace, punctuation
        and limited number of stop words relevant to the case. This
        could be generalized by using a larger set of stop words 
        i.e. scipy list.'''
        
    def strip_stop_words_and_punct(text):
        
        stop_words = [ 'etc','is', 'formerly', 'eg', 'if', 
                    'for', 'de','co', 'with', 'it', 
                    'to', 'an', 'inc', 'are', 'a', 
                    'of', 'as', 'ltd', 'the', 'ny', 'corp',
                    'in', 'its','nyc', 'dba', 'fund', 'new york']
        
        
        text = ' '.join([word for word in text.split() if word not in stop_words])
        text = ' '.join([c for c in text.split() if c not in string.punctuation])
        text = text.strip()
        return text
    
    for df, col in to_clean:
        df[col] = df[col].str.lower()
        df[col] = df[col].apply(lambda text: 
                                strip_stop_words_and_punct(str(text)))
    
        return df[col]    


########################################################################################################################

def database_reconciliation(database_one, name_reference, month):
    
    '''This function automates a manual process to reconicle two 
       leagcy databases with no API protocol. It is constructed to
       solve for inconsistent naming betweeen the databases by using a 
       third party list that was manually matched by hand.'''

    scrub_text_column([(database_one, 'Name')]) 
    
    type_one   =  database_one[database_one.Type.str.contains('Op')]
    type_two   =  database_one[database_one.Type.str.contains('Ch')]
    type_three =  database_one[database_one.Type.str.contains('Jo')]
    type_four  =  database_one[database_one.Type.str.contains('Me')]
    

        
    
    pages        = [[type_one,    'type_one'], 
                    [type_two,    'type_two'], 
                    [type_three,  'type_three'], 
                    [type_four,   'type_four']]


    month        = pd.ExcelWriter('{}_database_reconciliation.xlsx'.format(month), 
                                  engine ='xlsxwriter')
   
    for page in pages:
        
        df      = page[0]
        name    = page[1]

        # df_shape  = df.shape

        db2  = pd.read_csv('{}_database_two.csv'.format(name), \
                              usecols= ['Supplier Name', 
                                        'Actual Dollars']).groupby('Supplier Name')\
                                                            .sum()\
                                                            .rename(columns={'Actual Dollars':'Total DataBase2 $'})\
                                                            .reset_index()\
                                                            .merge( name_reference, 
                                                                    how='left')\
                                                                                .fillna('CHECK')
        
        scrub_text_column([(db2, 'Supplier Name')]) 
    
                                             
        df      = pd.merge( df, 
                            name_reference, 
                            how='left')\
                                        .fillna('CHECK')\
                                        .drop(columns =['Type'])
                                             
    
                                              
        reconciliation = pd.merge(db2,
                                  df, 
                                  how='outer', 
                                  on=['Supplier Name', 
                                      'DB1 Name'])\
                                                        .set_index(['Supplier Name',
                                                                    'Total DataBase2 $', 
                                                                    'DB1 Name'])
    
        reconciliation.to_excel(month, 
                                sheet_name = name)
        
    month.close()


########################################################################################################################

def fuzzy_database_reconciliation(database_one, month):
    
    ''' NOW DEFUNCT AS UNIQUE KEY WAS ADDED TO DB
        
        This reconciles two databases without API protocol and that lack
        matching keys utilizing fuzzy matches instead of a third reference 
        Dataframe. This is extremely slow.
        
        TO DO:
        
        - Instead of looping, append the db1 database_ones and add 'Type' 
          to multi-index 
        - Try using fuzzy matches to create a third dataframe of 
          merge on, then dropping the merge column.'''
    
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
   
    def fuzzreplace(name, df, col):
        answer = process.extractOne(name , df[col])
        
        if answer[1] > 95:
            name = answer[0]
        
        return name
     
    database_one = pd.read_excel(database_one)

    scrub_text_column([(database_one, 'Name')])
    
    type_one   =  database_one[database_one.Type.str.contains('Op')]
    type_two   =  database_one[database_one.Type.str.contains('Ch')]
    type_three =  database_one[database_one.Type.str.contains('Jo')]
    type_four  =  database_one[database_one.Type.str.contains('Me')]
    
    
    pages        = [[type_one,    'type_one'], 
                    [type_two,    'type_two'], 
                    [type_three,  'type_three'], 
                    [type_four,   'type_four']]


    month        = pd.ExcelWriter('{}_database_reconciliation.xlsx'.format(month),
                                  engine ='xlsxwriter')
    
    
    for page in pages:
        
        df             = page[0]
        name           = page[1]

        db2         = pd.read_csv('{}_database_two.csv'.format(name),
                                     usecols=['Supplier Name', 
                                              'Actual Dollars'])\
                                                                .groupby('Supplier Name')\
                                                                .sum()\
                                                                .rename(columns ={'Actual Dollars':'Total DB2 $'})\
                                                                .reset_index()
        
        scrub_text_column([(db2, 'Supplier Name')]) 

        df['Name'] = df['Name'].apply(lambda row: fuzzreplace(row, 
                                                              db2, 
                                                              'Supplier Name'))
        
        reconciliation = db2.merge(  df, 
                                        how='outer', 
                                        on=['Supplier Name', 
                                            'Name'])\
                                                    .set_index(['Supplier Name',
                                                                'Total DB2 $', 
                                                                'Name'])
        
        reconciliation.to_excel(month, 
                                sheet_name = name)
        
    month.close()


In [90]:
import pandas as pd
import string 

match_file = 'Database Name Match Reference.xlsx'
db1_file = 'database_reconciliation_pull.xlsx'

db1 = pd.read_excel(db1_file)\
                             .rename(columns ={'Name': 'db1 Name'})

name_reference = pd.read_excel()\
                                .rename(columns ={'DB2 Name': 'Supplier Name'})



database_reconciliation(db1, name_reference, 'October')
fuzzy_database_reconciliation(db1, 'October')